## Using ak_requests

In [1]:
import common as ak
import numpy as np
from scipy.optimize import linprog

%load_ext autoreload
%autoreload 2

In [2]:
item_dict = ak.get_item_dict(lang="en_US")
stage_dict = ak.get_stage_dict()
craft_dict = ak.get_craft_dict()
psdf = ak.get_pengstats_df(server="CN")

In [3]:
stage_san_cost, stage_names, stage_names_rev, stage_lmd_dict = ak.get_stage_info(psdf, stage_dict)
item_names, item_names_rev = ak.get_item_info(item_dict)
event_names, event_names_rev = ak.get_event_info(stage_names)

In [4]:
material_ids = ak.get_material_ids(item_names_rev, ak.MATERIAL_NAMES)
n_mats_all = len(material_ids)

In [5]:
#main_stage_ids = ak.get_main_stage_ids(psdf)
main_stage_ids = ak.get_main_and_perm_stage_ids(psdf)
stage_ids = np.concatenate((main_stage_ids,[
    "wk_melee_5",
    "wk_kc_5",
    "wk_fly_5"
]))

stage_san_all = ak.get_san_cost(stage_ids, stage_san_cost)
n_stages_all = len(stage_ids)

In [6]:
craft_matrix, subprod_matrix = ak.get_craft_matrix(craft_dict, material_ids)

In [7]:
drop_matrix_all, coeff_matrix_all = ak.get_drop_matrix(psdf, stage_ids, material_ids, stage_lmd_dict)

In [8]:
drop_matrix, stage_san = ak.filter_drop_matrix(drop_matrix_all, stage_san_all)

In [9]:
byprod_rate = 0.18
craft_constraint_matrix = ak.get_craft_constraint_matrix(craft_matrix, subprod_matrix, byprod_rate)

In [10]:
coeff = np.sum(drop_matrix, axis=0)
res = linprog(-coeff,
              A_ub=drop_matrix, b_ub=stage_san,
              A_eq=craft_constraint_matrix, b_eq=np.zeros(len(craft_constraint_matrix)),
              bounds=(0, None)
             )

print(res.success)
## Make dict from itemId -> msv at the end
msv = res.x
msv_dict = {}
for i, d in enumerate(material_ids):
    msv_dict[d] = msv[i]
    print("{}: {:.3f}".format(item_names[d], msv[i]))

True
LMD: 0.004
Pure Gold: 11.332
Drill Battle Record: 0.769
Frontline Battle Record: 1.539
Tactical Battle Record: 3.846
Strategic Battle Record: 7.693
Skill Summary - 1: 1.565
Skill Summary - 2: 3.980
Skill Summary - 3: 10.119
Orirock: 0.817
Orirock Cube: 2.526
Orirock Cluster: 12.441
Orirock Concentration: 46.418
Damaged Device: 3.074
Device: 9.297
Integrated Device: 36.999
Optimized Device: 87.967
Ester: 1.764
Polyester: 5.367
Polyester Pack: 21.281
Polyester Lump: 88.579
Sugar Substitute: 1.848
Sugar: 5.618
Sugar Pack: 22.285
Sugar Lump: 95.583
Oriron Shard: 2.266
Oriron: 6.872
Oriron Cluster: 27.301
Oriron Block: 109.536
Diketon: 2.264
Polyketon: 6.866
Aketon: 27.275
Keton Colloid: 100.546
Loxic Kohl: 22.090
White Horse Kohl: 77.830
Manganese Ore: 27.058
Manganese Trihydrate: 94.141
Grindstone: 29.432
Grindstone Pentahydrate: 90.386
RMA70-12: 36.802
RMA70-24: 85.612
Polymerization Preparation: 242.722
Bipolar Nanoflake: 229.848
D32 Steel: 256.360
Coagulating Gel: 30.567
Polymeriz

In [11]:
ak.print_craft_materials("Cutting Fluid Solution", craft_matrix, material_ids, item_names, item_names_rev)

LMD: 300
RMA70-12: 1
Crystalline Component: 1
Compound Cutting Fluid: 1


In [12]:
ak.print_stage_drops("R8-11", drop_matrix, stage_ids, material_ids, stage_names_rev, item_names)

LMD: 252
Crystalline Component: 52.71%
Orirock Cube: 36.70%
Orirock: 21.42%
Device: 13.91%
Pure Gold: 10.17%
Damaged Device: 7.94%
Orirock Cluster: 3.27%
Crystalline Circuit: 2.69%
Loxic Kohl: 2.66%
Coagulating Gel: 2.09%
Integrated Device: 1.82%


In [13]:
print(stage_names_rev["LS-5"])

wk_kc_5


In [14]:
print(item_names_rev["Pure Gold"])

3003


In [15]:
event_ids = ak.get_event_ids("NL", psdf, event_names_rev)
event_stage_san = ak.get_san_cost(event_ids, stage_san_cost)
event_drop_matrix, _ = ak.get_drop_matrix(psdf, event_ids, material_ids, stage_lmd_dict)
#sanity_return = np.matmul(event_drop_matrix, msv)
print(ak.get_stage_efficiency(event_drop_matrix, msv, event_stage_san))

[0.55341544 0.54279609 0.51266984 0.71364978 0.86720514 0.73884496
 0.88446222 1.06199691 0.93620687 1.00782741]
